In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import string
import collections
import numpy as np
import random
import tensorflow as tf
import math
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import sys
import argparse
import random
from tempfile import gettempdir
import zipfile

from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builti

from tensorflow.contrib.tensorboard.plugins import projector

In [3]:
current_path = os.path.dirname(os.path.realpath(sys.argv[0]))

parser = argparse.ArgumentParser()
parser.add_argument(
    '--log_dir',
    type=str,
    default=os.path.join(current_path, 'log'),
    help='The log directory for TensorBoard summaries.')
FLAGS, unparsed = parser.parse_known_args()

# Create the directory for TensorBoard variables if there is not.
if not os.path.exists(FLAGS.log_dir):
  os.makedirs(FLAGS.log_dir)


In [4]:
twentyng_data = fetch_20newsgroups(subset='train',remove=('headers','footers','quotes'))

In [5]:
tNG_data = twentyng_data.data

In [6]:
def remove_punctuation(s):
    tr = str.maketrans("", "", string.punctuation)
    ##s = "Hello! It is time to remove punctuations. It is easy, you will see."
    s = s.translate(tr)
    return s

In [7]:
#tNG_data

In [8]:
low = []
for line in tNG_data:
    line = line.strip() # skip the empty line
    line = line.replace('\n',' ') # replaces new lines to empty spaces
    #line = line.lstrip()
    line = remove_punctuation(line)
    lines=line.split()
    #print(lines)
    low.extend(lines)

In [9]:
vocabulary = low

In [10]:
vocabulary_size = 50000

#"""Process raw inputs into a dataset."""
def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [11]:
# Filling 4 global variables:
# data - list of codes (integers from 0 to vocabulary_size-1).
#   This is the original text but words are replaced by their codes
# count - map of words(strings) to count of occurrences
# dictionary - map of words(strings) to their codes(integers)
# reverse_dictionary - maps codes(integers) to words(strings)
data, count, dictionary, reverse_dictionary = build_dataset(
    vocabulary, vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0

Most common words (+UNK) [['UNK', 74139], ('the', 94602), ('to', 51811), ('of', 45906), ('a', 40446)]
Sample data [8, 23, 1225, 33, 144, 50, 42, 89, 10777, 46] ['I', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me']


In [12]:
len(count)

50000

In [13]:
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  if data_index + span > len(data):
    data_index = 0
  buffer.extend(data[data_index:data_index + span])
  data_index += span
  for i in range(batch_size // num_skips):
    context_words = [w for w in range(span) if w != skip_window]
    words_to_use = random.sample(context_words, num_skips)
    for j, context_word in enumerate(words_to_use):
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[context_word]
    if data_index == len(data):
      buffer.extend(data[0:span])
      data_index = span
    else:
      buffer.append(data[data_index])
      data_index += 1
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

In [14]:
batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
print(labels)
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]], '->', labels[i, 0],reverse_dictionary[labels[i, 0]])

[[1225]
 [   8]
 [  23]
 [  33]
 [1225]
 [ 144]
 [  50]
 [  33]]
23 was -> 1225 wondering
23 was -> 8 I
1225 wondering -> 23 was
1225 wondering -> 33 if
33 if -> 1225 wondering
33 if -> 144 anyone
144 anyone -> 50 out
144 anyone -> 33 if


In [15]:
# Step 4: Build and train a skip-gram model.

batch_size = 128
embedding_size = 300  # Dimension of the embedding vector.
skip_window = 1  # How many words to consider left and right.
num_skips = 2  # How many times to reuse an input to generate a label.
num_sampled = 64  # Number of negative examples to sample.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. These 3 variables are used only for
# displaying model accuracy, they don't affect calculation.
valid_size = 16  # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
#valid_examples = np.random.choice(50000, 50000, replace=False)

graph = tf.Graph()

with graph.as_default():

  # Input data.
  with tf.name_scope('inputs'):
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    with tf.name_scope('embeddings'):
        embeddings = tf.Variable(
          tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    with tf.name_scope('weights'):
        nce_weights = tf.Variable(
          tf.truncated_normal(
              [vocabulary_size, embedding_size],
              stddev=1.0 / math.sqrt(embedding_size)))
    with tf.name_scope('biases'):
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  # Explanation of the meaning of NCE loss:
  #   http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
  with tf.name_scope('loss'):
    loss = tf.reduce_mean(
        tf.nn.nce_loss(
            weights=nce_weights,
            biases=nce_biases,
            labels=train_labels,
            inputs=embed,
            num_sampled=num_sampled,
            num_classes=vocabulary_size))

  # Add the loss value as a scalar to summary.
  tf.summary.scalar('loss', loss)

  # Construct the SGD optimizer using a learning rate of 1.0.
  with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings,
                                            valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Merge all summaries.
  merged = tf.summary.merge_all()

  # Add variable initializer.
  init = tf.global_variables_initializer()

  # Create a saver.
  saver = tf.train.Saver()

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [16]:
# Step 5: Begin training.
num_steps = 100001

with tf.Session(graph=graph) as session:
  # Open a writer to write summaries.
  writer = tf.summary.FileWriter(FLAGS.log_dir, session.graph)

  # We must initialize all variables before we use them.
  init.run()
  print('Initialized')

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(batch_size, num_skips,
                                                skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # Define metadata variable.
    run_metadata = tf.RunMetadata()

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    # Also, evaluate the merged op to get all summaries from the returned "summary" variable.
    # Feed metadata variable to session for visualizing the graph in TensorBoard.
    _, summary, loss_val = session.run(
        [optimizer, merged, loss],
        feed_dict=feed_dict,
        run_metadata=run_metadata)
    average_loss += loss_val

    # Add returned summaries to writer in each step.
    writer.add_summary(summary, step)
    # Add metadata to visualize the graph for the last run.
    if step == (num_steps - 1):
      writer.add_run_metadata(run_metadata, 'step%d' % step)

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step ', step, ': ', average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = 'Nearest to %s:' % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = '%s %s,' % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()

  # Write corresponding labels for the embeddings.
  with open(FLAGS.log_dir + '/metadata.tsv', 'w') as f:
    for i in xrange(vocabulary_size):
      f.write(reverse_dictionary[i] + '\n')

  # Save the model for checkpoints.
  saver.save(session, os.path.join(FLAGS.log_dir, 'model.ckpt'))

  # Create a configuration for visualizing embeddings with the labels in TensorBoard.
  config = projector.ProjectorConfig()
  embedding_conf = config.embeddings.add()
  embedding_conf.tensor_name = embeddings.name
  embedding_conf.metadata_path = os.path.join(FLAGS.log_dir, 'metadata.tsv')
  projector.visualize_embeddings(writer, config)

writer.close()

Initialized
Average loss at step  0 :  291.2022705078125
Nearest to The: 17, CONSTRUED, suggestionscomments, underwear, microwave, supreme, decay, forgot,
Nearest to by: Matteau, watchers, FTPMAIL, Leverrier, barcoded, adept, Spacepac, bugger,
Nearest to get: parabolic, Towers, BILLION, coastlands, CSS, repairs, solemn, Roth,
Nearest to or: cease, FrequentlyAsked, Into, Kalamazoo, unveiled, withheld, FFL, Runtime,
Nearest to more: AllStar, XDMCP, Weight, Whosoever, CDIN, strawman, bindings, ministries,
Nearest to up: Kingsley, ALS, peaks, ASPI4DOS, 041893, Feigenbaum, chum, Tax,
Nearest to your: Deputy, divisional, mournful, works, ranged, Davis, WarsCivil, vs,
Nearest to who: xmosaic, 5k, directors, goin, AutoWeek, Edition, fates, pharmaceutical,
Nearest to X: particular, Spencer, deterministic, Automap, sermon, logia, MILITECH, grass,
Nearest to can: Drysdale, yourself, imperialist, Rasulovs, guarded, sovereignty, M514514514514514514514514514514514514514514514, cracked,
Nearest to wa

Average loss at step  58000 :  10.219114861667157
Average loss at step  60000 :  6.9278001153171065
Nearest to The: If, This, A, the, but, What, To, But,
Nearest to by: from, were, with, be, through, for, are, as,
Nearest to get: make, want, have, use, need, give, do, find,
Nearest to or: and, but, The, If, for, UNK, Or, about,
Nearest to more: less, longer, better, others, some, you, posted, no,
Nearest to up: off, UNK, back, them, out, us, just, him,
Nearest to your: my, their, his, the, our, her, My, these,
Nearest to who: He, They, that, What, We, almost, already, It,
Nearest to X: program, UNK, available, The, server, 3, possible, Windows,
Nearest to can: will, could, would, should, cant, may, didnt, wont,
Nearest to was: is, were, has, had, am, wasnt, seems, are,
Nearest to then: And, now, UNK, said, so, them, we, it,
Nearest to my: your, his, their, our, the, My, a, these,
Nearest to this: it, you, that, I, these, we, so, some,
Nearest to UNK: it, now, them, then, once, goes, is

In [16]:
# Step 6: Visualize the embeddings.


# pylint: disable=missing-docstring
# Function to draw visualization of distance between embeddings.
def plot_with_labels(low_dim_embs, labels, filename):
  assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
  plt.figure(figsize=(18, 18))  # in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x, y)
    plt.annotate(
        label,
        xy=(x, y),
        xytext=(5, 2),
        textcoords='offset points',
        ha='right',
        va='bottom')

  plt.savefig(filename)


try:
  # pylint: disable=g-import-not-at-top
  from sklearn.manifold import TSNE
  import matplotlib.pyplot as plt

  tsne = TSNE(
      perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
  plot_only = 500
  low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
  labels = [reverse_dictionary[i] for i in xrange(plot_only)]
  plot_with_labels(low_dim_embs, labels, os.path.join(gettempdir(), 'tsne.png'))

except ImportError as ex:
  print('Please install sklearn, matplotlib, and scipy to show embeddings.')
  print(ex)

In [17]:
gettempdir()

'/var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T'

In [18]:
print(reverse_dictionary[90])

In


In [17]:
from sklearn.metrics.pairwise import cosine_similarity
sim1 = cosine_similarity(final_embeddings)

In [18]:
valid_word = 'the'
key = 0
for k,v in reverse_dictionary.items():
    if v == valid_word:
        key = k
        break
i=key
top_k = 20  # number of nearest neighbors
print("i:",i)

nearest = (-sim1[i, :]).argsort()[1:top_k + 1]
log_str = 'Nearest to %s:' % valid_word
for k in xrange(top_k):
    close_word = reverse_dictionary[nearest[k]]
    log_str = '%s %s,' % (log_str, close_word)
print(log_str)

i: 1
Nearest to the: The, going, youre, these, even, and, did, people, common, you, only, probably, likely, obvious, think, but, its, yet, certainly, every,
